# Text mining assignment #1 
[교보문고 종합주간 베스트의 메타데이터 수집] <br>
`20185435 황재원`

- 순위, 소제목 (subtitle), 제목, 상세보기 링크, 이벤트 정보, 저자, 출판사, 출판일자, 가격, 추가 정보 
- 10 페이지 수집

In [1]:
## install library
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

import time
import re
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

In [2]:
# 접속하기
driver=webdriver.Chrome(r'/Users/hwangjaewon/Desktop/chromedriver')
driver.get("http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf")
time.sleep(2) 

In [3]:
rank=[]
subtitle=[]
title=[]
link=[]
price=[]
info=[]
author_list=[]
author=[]
pub_company=[]
pub_year=[]
event = []

In [4]:
#크롤링 함수 지정
def crawling(): 
    #beautifulsoup 활용하여 크롤링
    html= driver.page_source
    soup= BeautifulSoup(html, 'lxml')
    
    #순위
    con=soup.find('ul', {'class':'list_type01'}).findAll('div', {'class':'cover'})
    for b in con:
        rank.append(b.find('strong').text)
    
    #소제목, 제목, 상세보기 링크, 가격, 추가정보
    contents = soup.find('ul', {'class':'list_type01'}).findAll('div', {'class':'detail'})
    for b in contents:
        subtitle.append(b.find('div', {'class': 'subtitle'}).text.strip()) # 소제목
        title.append(b.find('div', {'class': 'title'}).find('strong').text)# 제목
        link.append(b.find('div', {'class': 'title'}).find('a')['href']) #상세보기 링크
        price.append(b.find('div', {'class': 'price'}).get_text(strip=True)) # 가격
        info.append(b.find('div', {'class': 'info'}).get_text(strip=True)) # 추가 정보

    #저자 #출판사 #출판일자
    remove_str = "저자 더보기"
    for i in soup.find_all('div', attrs= {'class': 'author'}):
        author_list.append(i.get_text(strip=True))
        for j in range(len(author_list)):
            word=''.join( x for x in author_list[j] if x not in remove_str)
            word= word.split('|')
        author.append(word[0])
        pub_company.append(word[1])
        pub_year.append(word[2])
    
    return rank,subtitle, title, link, price, info, author, pub_company, pub_year

In [5]:
def event_crawling():
    html= driver.page_source
    soup= BeautifulSoup(html, 'lxml')
    contents = soup.find('ul', {'class':'list_type01'}).findAll('div', {'class':'detail'})
    list_event=[b.find('span', {'class': 'icon'}).find_all('img') for b in contents]
    for x in range(len(list_event)) :
        list_length = len(list_event[x])
        x_event = []
        for j in range(list_length) :
            ele = list_event[x][j]['alt']
            x_event.append(ele)
        event.append(x_event)
    return event

In [6]:
#반복문 돌리면서 10페이지 크롤링 진행
for f in tqdm(range(2,11)):
    rank, subtitle, title, link, price, info, author, pub_company, pub_year=crawling()
    event = event_crawling()
    nextpage = driver.find_element_by_css_selector("#main_contents > div:nth-child(8) > div.list_paging > ul > li:nth-child({}) > a".format(f)) 
    nextpage.click()
    
#마지막 페이지 크롤링
rank,subtitle, title, link, price, info, author, pub_company, pub_year=crawling() 
event = event_crawling()
driver.close()

100%|██████████| 9/9 [00:14<00:00,  1.65s/it]


In [8]:
bestseller=pd.DataFrame({'순위':rank,'부제목':subtitle,'제목':title, '상세보기링크':link, '이벤트 정보':event,
                         '가격':price,'추가정보':info, '저자':author, '출판사':pub_company, '출판연도':pub_year})
bestseller
bestseller.to_csv("교보문고 베스트셀러 크롤링.csv")

,순위,부제목,제목,상세보기링크,이벤트 정보,가격,추가정보,저자,출판사,출판연도
0,1,서울대 소비트렌드 분석센터의 2022 전망,트렌드 코리아 2022,http://www.kyobobook.co.kr/product/detailViewK...,"[무료배송, 이벤트, 오늘의책, 소득공제]","도서16,200원[10%↓ +5% 적립]|eBook12,600원","지금 주문하면내일(22일,금)도착 예정입니다",김난도전미영최지혜이향은이준영,미래의창,2021년10월06일
1,2,아주 작은 날갯짓의 시작,인생은 실전이다,http://www.kyobobook.co.kr/product/detailViewK...,"[무료배송, 소득공제]","도서16,650원[10%↓ +5% 적립]|eBook12,900원","지금 주문하면내일(22일,금)도착 예정입니다",주언규신영준,상상스퀘어,2021년09월08일
2,3,철학이 우리 인생에 스며드는 순간,소크라테스 익스프레스,http://www.kyobobook.co.kr/product/detailViewK...,"[무료배송, 이벤트, 오늘의책, sam, 소득공제]","도서16,200원[10%↓ +5% 적립]|eBook12,500원","지금 주문하면내일(22일,금)도착 예정입니다",에릭와이너,어크로스,2021년04월28일
3,4,단골손님을 찾습니다,달러구트 꿈 백화점. 2,http://www.kyobobook.co.kr/product/detailViewK...,"[무료배송, 이벤트, 오늘의책, 소득공제]","도서12,420원[10%↓ +5% 적립]|eBook9,660원","지금 주문하면내일(22일,금)도착 예정입니다",이미예,팩토리나인,2021년07월27일
4,5,주문하신 꿈은 매진입니다,달러구트 꿈 백화점,http://www.kyobobook.co.kr/product/detailViewK...,"[무료배송, 이벤트, MD의 선택, sam, 소득공제]","도서12,420원[10%↓ +5% 적립]|eBook9,100원","지금 주문하면내일(22일,금)도착 예정입니다",이미예,팩토리나인,2020년07월08일
...,...,...,...,...,...,...,...,...,...,...
194,196,,순례자의 길(개정판 2판)(기독교 영성 시리즈),http://www.kyobobook.co.kr/product/detailViewK...,"[무료배송, sam, 소득공제]","도서13,500원[10%↓ +5% 적립]|eBook15,000원","지금 주문하면내일(22일,금)도착 예정입니다",무명의순례,은성,2003년10월09일
195,197,,열두 살 장래 희망(양장본 HardCover),http://www.kyobobook.co.kr/product/detailViewK...,"[무료배송, 오늘의책, 소득공제]","도서11,700원[10%↓ +5% 적립]","지금 주문하면내일(22일,금)도착 예정입니다",박성우,창비,2021년07월23일
196,198,본질에 집중하는 힘,에센셜리즘,http://www.kyobobook.co.kr/product/detailViewK...,"[무료배송, 이벤트, 오늘의책, sam, 소득공제]","도서13,500원[10%↓ +5% 적립]|eBook10,500원","지금 주문하면내일(22일,금)도착 예정입니다",그렉맥커운,알에이치코리아,2014년09월19일
197,199,ETF만으로도 꼬박꼬박 연 40% 수익 내는 법,부의 시작,http://www.kyobobook.co.kr/product/detailViewK...,"[무료배송, 이벤트, MD의 선택, 소득공제]","도서18,900원[10%↓ +5% 적립]","지금 주문하면내일(22일,금)도착 예정입니다",박민수,길벗,2021년10월03일


확인 결과 160번째 정보만 크롤링되지 않음(이유 불명).